In [1]:
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

import pickle
import numpy as np
import timeit
from load_cifar import *
import time

In [18]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [19]:
#learning rate
lr = .01
#number of traning steps
num_steps =50 # longer fitting model cycles 
#number of batch_size
batch_size = 64 # 128*5 - more training data 

#network parameters
n_hidden_1 = 100 # 100 - reduce over fitting 
n_hidden_2 = 100
n_hidden_3 = 100
num_input = 32*32*3 # num of dimension of an image 
num_classes = 10 # num of label names 

dropout_rate = 0.3

<h1>Define Placeholder</h1>

In [20]:
#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.float32,[None,num_classes],name='Y')
training = tf.placeholder_with_default(False, shape=(),name='training')

<h1>Define Neural Network Architecture</h1>

In [21]:
# give l2 regularizer to weights 
regularizer = tf.contrib.layers.l2_regularizer(0.1)
# give batch normalization: by subtracting the batch mean and dividing by the batch standard deviation.

#define a neural net model
def neural_net(x):
    layer_1_out = tf.layers.dense(x,n_hidden_1,activation=None,kernel_initializer=tf.random_normal_initializer(), kernel_regularizer=regularizer)
    layer_1_act = tf.nn.relu(layer_1_out)        
    layer_1_bn = tf.layers.batch_normalization(layer_1_act,momentum=0.99,training=training)
    
    layer_2_drop = tf.layers.dropout(layer_1_bn ,dropout_rate,training=training)
    layer_2_out = tf.layers.dense( layer_2_drop, n_hidden_2,activation=None,kernel_initializer=tf.random_normal_initializer(),kernel_regularizer=regularizer)
    layer_2_act = tf.nn.relu(layer_2_out)        
    layer_2_drop = tf.layers.dropout(layer_2_act ,dropout_rate,training=training)
    layer_2_bn = tf.layers.batch_normalization(layer_2_act,momentum=0.99,training=training)
    
    layer_3_drop = tf.layers.dropout(layer_2_bn ,dropout_rate,training=training)
    layer_3_out = tf.layers.dense( layer_3_drop, n_hidden_3,activation=None,kernel_initializer=tf.random_normal_initializer(),kernel_regularizer=regularizer)
    layer_3_act = tf.nn.relu(layer_3_out)        
    layer_3_drop = tf.layers.dropout(layer_3_act ,dropout_rate,training=training)
    layer_3_bn = tf.layers.batch_normalization(layer_3_act,momentum=0.99,training=training)
    
    out = tf.layers.dense(layer_3_bn,num_classes,activation=None,kernel_initializer=tf.random_normal_initializer(),kernel_regularizer=regularizer)
            
    return out

<h1>Define cost and optimization</h1>

In [22]:
#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y),name='loss')
l2_loss = tf.losses.get_regularization_loss()
loss+=l2_loss

#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))  

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [23]:
valid_x , valid_y = load_preprocessed_validation_batch()
test_x , test_y = load_preprocessed_test_batch(test_mini_batch_size=batch_size)

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        start = time.time()
        for batch_id in range(1,6):        
            batch_x, batch_y = load_preprocessed_training_batch( batch_id=batch_id, mini_batch_size = batch_size)
            #run optimization            
            sess.run([train_op,extra_update_ops], feed_dict={X:batch_x, Y:batch_y,training:True})        
            
        end = time.time()
        acc = sess.run(accuracy,feed_dict={X:valid_x, Y:valid_y})
        if i%10==0:
            print("step "+str(i)+", validation accuracy= {:.3f}".format(acc) , '..time used:', round( end -start, 4) )
    
    print("Training finished!")
    
    print("Testing accuracy:", sess.run(accuracy, feed_dict={X: test_x, Y: test_y}))

step 0, validation accuracy= 0.092 ..time used: 2.5051
step 10, validation accuracy= 0.138 ..time used: 1.5841
step 20, validation accuracy= 0.182 ..time used: 1.7525
step 30, validation accuracy= 0.146 ..time used: 1.5621
step 40, validation accuracy= 0.099 ..time used: 1.6831
Training finished!
Testing accuracy: 0.078125


## Finding and comment

Unfortunately, after 50 epochs, I am not able to get a good result of accuracy on the validation dataset. Basically, my full connected neural network is not learning much. I am not sure why this happened. 